<a href="https://colab.research.google.com/github/rianeps/Machine-Learning-Assignment/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Classification and Regression
Author: Nepoliyan Ria

# FARS Accident Severity Classification

This notebook performs EDA, preprocessing, and model evaluation on the FARS dataset. The target variable is the last column in the CSV (accident severity).

**Goals**
- Exploratory data analysis (EDA) with interactive Plotly visuals.
- Robust preprocessing (missing values, categorical encoding, scaling).
- Cross-validated evaluation.
- Four distinct ML pipelines with hyperparameter tuning.


In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_validate, cross_val_predict
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (classification_report, confusion_matrix,
                             ConfusionMatrixDisplay, roc_auc_score)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

pd.set_option("display.max_columns", 200)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
# Load data
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Machine Learning Assignment/fars.csv"

df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
df.head()


Shape: (100968, 30)


,CASE_STATE,AGE,SEX,PERSON_TYPE,SEATING_POSITION,RESTRAINT_SYSTEM-USE,AIR_BAG_AVAILABILITY/DEPLOYMENT,EJECTION,EJECTION_PATH,EXTRICATION,NON_MOTORIST_LOCATION,POLICE_REPORTED_ALCOHOL_INVOLVEMENT,METHOD_ALCOHOL_DETERMINATION,ALCOHOL_TEST_TYPE,ALCOHOL_TEST_RESULT,POLICE-REPORTED_DRUG_INVOLVEMENT,METHOD_OF_DRUG_DETERMINATION,DRUG_TEST_TYPE_(1_of_3),DRUG_TEST_RESULTS_(1_of_3),DRUG_TEST_TYPE_(2_of_3),DRUG_TEST_RESULTS_(2_of_3),DRUG_TEST_TYPE_(3_of_3),DRUG_TEST_RESULTS_(3_of_3),HISPANIC_ORIGIN,TAKEN_TO_HOSPITAL,RELATED_FACTOR_(1)-PERSON_LEVEL,RELATED_FACTOR_(2)-PERSON_LEVEL,RELATED_FACTOR_(3)-PERSON_LEVEL,RACE,INJURY_SEVERITY
0,Alabama,34,Male,Driver,Front_Seat_-_Left_Side_(Drivers_Side),None_Used/Not_Applicable,Air_Bag_Available_but_Not_Deployed_for_this_Seat,Totally_Ejected,Unknown,Not_Extricated,Not_Applicable_-_Vehicle_Occupant,Yes_(Alcohol_Involved),Not_Reported,Whole_Blood,97,Reported_Unknown,Not_Reported,Unknown_if_Tested_for_Drugs,999,Not_Tested_for_Drugs,0,Not_Tested_for_Drugs,0,Non-Hispanic,No,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,White,Fatal_Injury
1,Alabama,20,Male,Driver,Front_Seat_-_Left_Side_(Drivers_Side),None_Used/Not_Applicable,Deployed_Air_Bag_from_Front,Totally_Ejected,Unknown,Not_Extricated,Not_Applicable_-_Vehicle_Occupant,No_(Alcohol_Not_Involved),Not_Reported,Not_Tested_for_Alcohol,96,No_Drugs,Not_Reported,Not_Tested_for_Drugs,0,Not_Tested_for_Drugs,0,Not_Tested_for_Drugs,0,Non-Hispanic,No,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,White,Fatal_Injury
2,Alabama,43,Male,Driver,Front_Seat_-_Left_Side_(Drivers_Side),Lap_and_Shoulder_Belt,Air_Bag_Not_Available_for_this_Seat,Not_Ejected,Not_Ejected/Not_Applicable,Extricated,Not_Applicable_-_Vehicle_Occupant,No_(Alcohol_Not_Involved),Not_Reported,Not_Tested_for_Alcohol,96,No_Drugs,Not_Reported,Not_Tested_for_Drugs,0,Not_Tested_for_Drugs,0,Not_Tested_for_Drugs,0,Non-Hispanic,No,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,Black,Fatal_Injury
3,Alabama,38,Female,Passenger_of_a_Motor_Vehicle_in_Transport,Front_Seat_-_Right_Side,Lap_and_Shoulder_Belt,Air_Bag_Not_Available_for_this_Seat,Not_Ejected,Not_Ejected/Not_Applicable,Extricated,Not_Applicable_-_Vehicle_Occupant,Not_reported,Not_Reported,Not_Tested_for_Alcohol,96,Not_Reported,Not_Reported,Not_Tested_for_Drugs,0,Not_Tested_for_Drugs,0,Not_Tested_for_Drugs,0,Not_a_Fatality_(Not_Applicable),Yes,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_a_Fatality_(Not_Applicable),Incapaciting_Injury
4,Alabama,50,Male,Driver,Front_Seat_-_Left_Side_(Drivers_Side),Lap_and_Shoulder_Belt,Deployed_Air_Bag_from_Front,Not_Ejected,Not_Ejected/Not_Applicable,Not_Extricated,Not_Applicable_-_Vehicle_Occupant,Yes_(Alcohol_Involved),Not_Reported,Whole_Blood,97,Reported_Unknown,Not_Reported,Unknown_if_Tested_for_Drugs,999,Not_Tested_for_Drugs,0,Not_Tested_for_Drugs,0,Non-Hispanic,Yes,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,Not_Applicable_-_Driver/None_-_All_Other_Persons,Black,Fatal_Injury


## EDA


In [ ]:
# Basic info
print(df.info())
print("\nMissing values (top 20):")
missing = df.isna().sum().sort_values(ascending=False)
print(missing.head(20))

# Target distribution
TARGET_COL = df.columns[-1]
class_counts = df[TARGET_COL].value_counts().reset_index()
class_counts.columns = [TARGET_COL, "count"]

fig = px.bar(class_counts, x=TARGET_COL, y="count", title="Class Distribution")
fig.show()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100968 entries, 0 to 100967
Data columns (total 30 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   CASE_STATE                           100968 non-null  object
 1   AGE                                  100968 non-null  int64 
 2   SEX                                  100968 non-null  object
 3   PERSON_TYPE                          100968 non-null  object
 4   SEATING_POSITION                     100968 non-null  object
 5   RESTRAINT_SYSTEM-USE                 100968 non-null  object
 6   AIR_BAG_AVAILABILITY/DEPLOYMENT      100968 non-null  object
 7   EJECTION                             100968 non-null  object
 8   EJECTION_PATH                        100968 non-null  object
 9   EXTRICATION                          100968 non-null  object
 10  NON_MOTORIST_LOCATION                100968 non-null  object
 11  POLICE_REPORTED_ALCOHOL_IN

In [ ]:
# Missing values by column (top 25)
missingdf = missing[missing > 0].head(25).reset_index()
missingdf.columns = ["feature", "missing_count"]

if not missingdf.empty:
    fig = px.bar(missingdf, x="feature", y="missing_count",
                 title="Top Missing Values by Feature",
                 text="missing_count")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()


In [ ]:
# Identify feature types
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numeric features:", len(numeric_features))
print("Categorical features:", len(categorical_features))


Numeric features: 5
Categorical features: 24


In [ ]:
# Numeric distributions (sampled for speed)
if numeric_features:
    sample = df.sample(min(5000, len(df)), random_state=RANDOM_STATE)
    fig = px.histogram(sample, x=numeric_features[0], nbins=50,
                       title=f"Distribution of {numeric_features[0]} (sampled)")
    fig.show()

# Correlation heatmap for numeric features
if len(numeric_features) >= 2:
    corr = df[numeric_features].corr()
    fig = px.imshow(corr, title="Correlation Heatmap (Numeric Features)")
    fig.show()


In [ ]:
# Categorical cardinalities (top 20)
if categorical_features:
    cardinality = df[categorical_features].nunique().sort_values(ascending=False)
    top_card = cardinality.head(20).reset_index()
    top_card.columns = ["feature", "unique_values"]
    fig = px.bar(top_card, x="feature", y="unique_values",
                 title="Top Categorical Cardinalities",
                 text="unique_values")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()


## Preprocessing
We use a `ColumnTransformer` to handle numeric and categorical features separately:
- Numeric: median imputation + scaling
- Categorical: most-frequent imputation + one-hot encoding


In [ ]:
# Compatibility helper for OneHotEncoder across sklearn versions
try:
    _ = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    onehot = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
except TypeError:
    onehot = OneHotEncoder(sparse=False, handle_unknown="ignore")

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", onehot)
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)


In [ ]:
# Sample for tuning if dataset is very large
if len(df) > 50000:
    tune_df = df.sample(50000, random_state=RANDOM_STATE, replace=False)
else:
    tune_df = df.copy()

X_tune = tune_df.drop(columns=[TARGET_COL])
y_tune = tune_df[TARGET_COL]


## Pipelines


In [ ]:
# CV config for this model
cv_full = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring_basic = {
    "accuracy": "accuracy",
    "balanced_accuracy": "balanced_accuracy",
    "f1_macro": "f1_macro"
}

# Model 1: Logistic Regression
pipe_lr = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=2000, solver="saga",
                               class_weight="balanced", n_jobs=-1))
])

param_distributions_lr = {
    "clf__C": np.logspace(-3, 2, 10),
    "clf__penalty": ["l2"],
}

search_lr = RandomizedSearchCV(
    pipe_lr,
    param_distributions=param_distributions_lr,
    n_iter=8,
    scoring="f1_macro",
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE),
    random_state=RANDOM_STATE,
    n_jobs=-1
)
search_lr.fit(X_tune, y_tune)
best_lr = search_lr.best_estimator_
print("LogReg best params:", search_lr.best_params_)

scores_lr = cross_validate(best_lr, X, y, cv=cv_full, scoring=scoring_basic, n_jobs=-1)
print(f"LogReg accuracy: {scores_lr['test_accuracy'].mean():.4f} ± {scores_lr['test_accuracy'].std():.4f}")
print(f"LogReg balanced accuracy: {scores_lr['test_balanced_accuracy'].mean():.4f}")
print(f"LogReg macro F1: {scores_lr['test_f1_macro'].mean():.4f}")


LogReg best params: {'clf__penalty': 'l2', 'clf__C': np.float64(27.825594022071257)}


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



LogReg accuracy: 0.7227 ± 0.0436
LogReg balanced accuracy: 0.5835
LogReg macro F1: 0.4564


In [ ]:
# CV config for this model
cv_full = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring_basic = {
    "accuracy": "accuracy",
    "balanced_accuracy": "balanced_accuracy",
    "f1_macro": "f1_macro"
}

# Model 2: Calibrated Linear SVM
pipe_svm = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", CalibratedClassifierCV(
        estimator=LinearSVC(class_weight="balanced", max_iter=5000),
        method="sigmoid",
        cv=3
    ))
])

param_distributions_svm = {
    "clf__estimator__C": np.logspace(-3, 2, 10),
}

search_svm = RandomizedSearchCV(
    pipe_svm,
    param_distributions=param_distributions_svm,
    n_iter=8,
    scoring="f1_macro",
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE),
    random_state=RANDOM_STATE,
    n_jobs=-1
)
search_svm.fit(X_tune, y_tune)
best_svm = search_svm.best_estimator_
print("CalibSVM best params:", search_svm.best_params_)

scores_svm = cross_validate(best_svm, X, y, cv=cv_full, scoring=scoring_basic, n_jobs=-1)
print(f"CalibSVM accuracy: {scores_svm['test_accuracy'].mean():.4f} ± {scores_svm['test_accuracy'].std():.4f}")
print(f"CalibSVM balanced accuracy: {scores_svm['test_balanced_accuracy'].mean():.4f}")
print(f"CalibSVM macro F1: {scores_svm['test_f1_macro'].mean():.4f}")


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning:


8 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **las

CalibSVM best params: {'clf__estimator__C': np.float64(27.825594022071257)}
CalibSVM accuracy: 0.7944 ± 0.0018
CalibSVM balanced accuracy: 0.4702
CalibSVM macro F1: 0.4790


In [ ]:
# CV config for this model
cv_full = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring_basic = {
    "accuracy": "accuracy",
    "balanced_accuracy": "balanced_accuracy",
    "f1_macro": "f1_macro"
}

# Model 3: Random Forest
pipe_rf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", RandomForestClassifier(
        n_estimators=300,
        class_weight="balanced_subsample",
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])

param_distributions_rf = {
    "clf__n_estimators": [200, 300, 500],
    "clf__max_depth": [None, 10, 20],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4]
}

search_rf = RandomizedSearchCV(
    pipe_rf,
    param_distributions=param_distributions_rf,
    n_iter=8,
    scoring="f1_macro",
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE),
    random_state=RANDOM_STATE,
    n_jobs=-1
)
search_rf.fit(X_tune, y_tune)
best_rf = search_rf.best_estimator_
print("RandForest best params:", search_rf.best_params_)

scores_rf = cross_validate(best_rf, X, y, cv=cv_full, scoring=scoring_basic, n_jobs=-1)
print(f"RandForest accuracy: {scores_rf['test_accuracy'].mean():.4f} ± {scores_rf['test_accuracy'].std():.4f}")
print(f"RandForest balanced accuracy: {scores_rf['test_balanced_accuracy'].mean():.4f}")
print(f"RandForest macro F1: {scores_rf['test_f1_macro'].mean():.4f}")


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



RandForest best params: {'clf__n_estimators': 300, 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 2, 'clf__max_depth': None}
RandForest accuracy: 0.7949 ± 0.0034
RandForest balanced accuracy: 0.6078
RandForest macro F1: 0.5466


In [ ]:
# CV config for this model
cv_full = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring_basic = {
    "accuracy": "accuracy",
    "balanced_accuracy": "balanced_accuracy",
    "f1_macro": "f1_macro"
}

# Model 4: Gradient Boosting + SelectKBest
pipe_gb = Pipeline(steps=[
    ("preprocess", preprocess),
    ("select", SelectKBest(mutual_info_classif, k=200)),
    ("clf", GradientBoostingClassifier(random_state=RANDOM_STATE))
])

param_distributions_gb = {
    "select__k": [100, 200, 400],
    "clf__n_estimators": [100, 200],
    "clf__learning_rate": [0.05, 0.1, 0.2],
    "clf__max_depth": [3, 5]
}

search_gb = RandomizedSearchCV(
    pipe_gb,
    param_distributions=param_distributions_gb,
    n_iter=8,
    scoring="f1_macro",
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE),
    random_state=RANDOM_STATE,
    n_jobs=-1
)
search_gb.fit(X_tune, y_tune)
best_gb = search_gb.best_estimator_
print("GradBoost best params:", search_gb.best_params_)

scores_gb = cross_validate(best_gb, X, y, cv=cv_full, scoring=scoring_basic, n_jobs=-1)
print(f"GradBoost accuracy: {scores_gb['test_accuracy'].mean():.4f} ± {scores_gb['test_accuracy'].std():.4f}")
print(f"GradBoost balanced accuracy: {scores_gb['test_balanced_accuracy'].mean():.4f}")
print(f"GradBoost macro F1: {scores_gb['test_f1_macro'].mean():.4f}")


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



GradBoost best params: {'select__k': 200, 'clf__n_estimators': 200, 'clf__max_depth': 3, 'clf__learning_rate': 0.1}
GradBoost accuracy: 0.8003 ± 0.0014
GradBoost balanced accuracy: 0.5489
GradBoost macro F1: 0.5480


In [ ]:
# Collect best models for later comparison
best_models = {
    "LogReg": best_lr,
    "CalibSVM": best_svm,
    "RandForest": best_rf,
    "GradBoost+SelectK": best_gb
}


In [ ]:

perf_df = pd.DataFrame({
    "model": ["GradBoost+SelectK", "RandForest", "CalibSVM", "LogReg"],
    "accuracy_mean": [0.8003, 0.7949, 0.7944, 0.7227],
    "accuracy_std":  [0.0014, 0.0034, 0.0018, 0.0436],
    "balanced_acc_mean": [0.5489, 0.6078, 0.4702, 0.5835],
    "f1_macro_mean": [0.5480, 0.5466, 0.4790, 0.4564]
}).sort_values("f1_macro_mean", ascending=False)

perf_df


,model,accuracy_mean,accuracy_std,balanced_acc_mean,f1_macro_mean
0,GradBoost+SelectK,0.8003,0.0014,0.5489,0.5480
1,RandForest,0.7949,0.0034,0.6078,0.5466
2,CalibSVM,0.7944,0.0018,0.4702,0.4790
3,LogReg,0.7227,0.0436,0.5835,0.4564


In [ ]:
perf_long = perf_df.melt(
    id_vars="model",
    value_vars=["accuracy_mean", "balanced_acc_mean", "f1_macro_mean"],
    var_name="metric",
    value_name="score"
)

fig = px.bar(
    perf_long,
    x="model",
    y="score",
    color="metric",
    barmode="group",
    title="Task 1 Classification: Cross-Validation Performance Comparison"
)
fig.update_layout(yaxis_title="Score", xaxis_title="Model")
fig.show()


In [ ]:
fig = px.bar(
    perf_df,
    x="model",
    y="accuracy_mean",
    error_y="accuracy_std",
    title="Accuracy (CV Mean ± Std)"
)
fig.update_layout(yaxis_title="Accuracy", xaxis_title="Model")
fig.show()


## Dataset 2: Regression

In this section we predict the growth-rate parameters (`a`, `mu`, `tau`, `a0`) from the experimental conditions.


In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_validate, cross_val_predict
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (classification_report, confusion_matrix,
                             ConfusionMatrixDisplay, roc_auc_score,
                             mean_absolute_error, mean_squared_error, r2_score)
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.svm import LinearSVC, SVR
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor
)
from sklearn.feature_selection import SelectKBest, mutual_info_classif

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

pd.set_option("display.max_columns", 200)


In [ ]:
# Load regression dataset
reg_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning Assignment/fitting-results.csv')

reg_df.head()


,n_cyanos,co2,light,SucRatio,Nsample,a,mu,tau,a0
0,262,73.578860,0.367894,0.451505,13123,5.033993,5.483011,0.552466,1.0
1,610,74.582194,0.481606,0.969900,12474,1.018406,1.830727,0.156458,1.0
2,118,33.111037,0.625418,0.909699,8867,2.569159,3.330662,0.412541,1.0
3,159,8.027676,0.575251,0.137124,16494,1.837341,3.371570,0.260798,1.0
4,957,38.796599,0.939799,0.234114,10267,0.999268,2.059303,0.098619,1.0


In [ ]:
# Basic EDA
reg_df.info()
reg_df.isna().sum()

# Target distributions
target_cols = ["a", "mu", "tau", "a0"]
for col in target_cols:
    fig = px.histogram(reg_df, x=col, nbins=40, title=f"{col} distribution")
    fig.show()

# Correlation overview
feature_cols = [c for c in reg_df.columns if c not in target_cols]
corr = reg_df[feature_cols + target_cols].corr(numeric_only=True)
fig = px.imshow(corr, text_auto=True, title="Feature/target correlation")
fig.show()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19992 entries, 0 to 19991
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   n_cyanos  19992 non-null  int64  
 1   co2       19992 non-null  float64
 2   light     19992 non-null  float64
 3   SucRatio  19992 non-null  float64
 4   Nsample   19992 non-null  int64  
 5   a         19992 non-null  float64
 6   mu        19992 non-null  float64
 7   tau       19992 non-null  float64
 8   a0        19992 non-null  float64
dtypes: float64(7), int64(2)
memory usage: 1.4 MB


In [ ]:
# Train/validation/test split (60/20/20) with shared indices
idx = np.arange(len(reg_df))
idx_train, idx_temp = train_test_split(idx, test_size=0.4, random_state=RANDOM_STATE)
idx_val, idx_test = train_test_split(idx_temp, test_size=0.5, random_state=RANDOM_STATE)

X = reg_df[feature_cols]

# Preprocessing: impute + scale numeric, impute + one-hot categorical
num_features = X.select_dtypes(include=["number"]).columns
cat_features = X.select_dtypes(exclude=["number"]).columns

num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess_reg = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_features),
        ("cat", cat_pipe, cat_features)
    ],
    remainder="drop"
)

cv_reg = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)


In [ ]:
# Model pipelines and hyperparameter search
pipe_ridge = Pipeline(steps=[
    ("preprocess", preprocess_reg),
    ("reg", Ridge())
])
param_dist_ridge = {
    "reg__alpha": np.logspace(-3, 3, 30)
}

pipe_svr = Pipeline(steps=[
    ("preprocess", preprocess_reg),
    ("reg", SVR())
])
param_dist_svr = {
    "reg__C": np.logspace(-2, 3, 20),
    "reg__gamma": ["scale", "auto"],
    "reg__epsilon": np.linspace(0.01, 1.0, 15)
}

pipe_rf = Pipeline(steps=[
    ("preprocess", preprocess_reg),
    ("reg", RandomForestRegressor(
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])
param_dist_rf = {
    "reg__n_estimators": [200, 400, 600],
    "reg__max_depth": [None, 5, 10, 20],
    "reg__min_samples_split": [2, 5, 10],
    "reg__min_samples_leaf": [1, 2, 4]
}

pipe_gb = Pipeline(steps=[
    ("preprocess", preprocess_reg),
    ("reg", GradientBoostingRegressor(random_state=RANDOM_STATE))
])
param_dist_gb = {
    "reg__n_estimators": [100, 200, 300],
    "reg__learning_rate": [0.01, 0.05, 0.1],
    "reg__max_depth": [2, 3, 4],
    "reg__subsample": [0.7, 0.9, 1.0]
}

search_results = {}

for target in target_cols:
    y_train = reg_df.loc[idx_train, target]
    X_train = X.iloc[idx_train]

    search_ridge = RandomizedSearchCV(
        estimator=pipe_ridge,
        param_distributions=param_dist_ridge,
        n_iter=20,
        scoring="neg_root_mean_squared_error",
        cv=cv_reg,
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    search_ridge.fit(X_train, y_train)
    search_results[(target, "Ridge")] = search_ridge

    search_svr = RandomizedSearchCV(
        estimator=pipe_svr,
        param_distributions=param_dist_svr,
        n_iter=20,
        scoring="neg_root_mean_squared_error",
        cv=cv_reg,
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    search_svr.fit(X_train, y_train)
    search_results[(target, "SVR")] = search_svr

    search_rf = RandomizedSearchCV(
        estimator=pipe_rf,
        param_distributions=param_dist_rf,
        n_iter=20,
        scoring="neg_root_mean_squared_error",
        cv=cv_reg,
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    search_rf.fit(X_train, y_train)
    search_results[(target, "RandomForest")] = search_rf

    search_gb = RandomizedSearchCV(
        estimator=pipe_gb,
        param_distributions=param_dist_gb,
        n_iter=20,
        scoring="neg_root_mean_squared_error",
        cv=cv_reg,
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    search_gb.fit(X_train, y_train)
    search_results[(target, "GradientBoosting")] = search_gb


In [ ]:
# Evaluate on validation and test sets

def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

results = []

for (target, name), search in search_results.items():
    X_val = X.iloc[idx_val]
    X_test = X.iloc[idx_test]
    y_val = reg_df.loc[idx_val, target]
    y_test = reg_df.loc[idx_test, target]

    val_pred = search.best_estimator_.predict(X_val)
    test_pred = search.best_estimator_.predict(X_test)

    val_mae, val_rmse, val_r2 = regression_metrics(y_val, val_pred)
    test_mae, test_rmse, test_r2 = regression_metrics(y_test, test_pred)

    results.append({
        "target": target,
        "model": name,
        "split": "val",
        "mae": val_mae,
        "rmse": val_rmse,
        "r2": val_r2
    })
    results.append({
        "target": target,
        "model": name,
        "split": "test",
        "mae": test_mae,
        "rmse": test_rmse,
        "r2": test_r2
    })

results_df = pd.DataFrame(results)
results_df


,target,model,split,mae,rmse,r2
0,a,Ridge,val,5.816120,127.753372,0.284876
1,a,Ridge,test,5.940504,137.435667,0.277500
2,a,SVR,val,1.444042,30.014763,0.831987
3,a,SVR,test,1.503553,37.604398,0.802314
4,a,RandomForest,val,0.450044,4.855486,0.972820
5,a,RandomForest,test,0.448206,5.625659,0.970426
6,a,GradientBoosting,val,0.550960,4.468411,0.974987
7,a,GradientBoosting,test,0.546334,5.165620,0.972844
8,mu,Ridge,val,4.189900,66.736545,0.301376
9,mu,Ridge,test,4.300338,72.657679,0.289961


In [ ]:
# Best model per target (by validation RMSE)
val_results = results_df[results_df["split"] == "val"]
val_best = val_results.sort_values(["target", "rmse"]).groupby("target").head(1)

# Join with test metrics for those best models
test_results = results_df[results_df["split"] == "test"]
best_summary = val_best.merge(
    test_results,
    on=["target", "model"],
    suffixes=("_val", "_test")
)

best_summary


,target,model,split_val,mae_val,rmse_val,r2_val,split_test,mae_test,rmse_test,r2_test
0,a,GradientBoosting,val,0.550960,4.468411,0.974987,test,0.546334,5.165620,0.972844
1,a0,RandomForest,val,0.003861,0.000083,0.700155,test,0.004125,0.000091,0.703066
2,mu,GradientBoosting,val,0.500209,2.476298,0.974077,test,0.519958,3.159090,0.969128
3,tau,RandomForest,val,0.019496,0.001121,0.993653,test,0.020769,0.001899,0.989301


In [ ]:


rmse_plot_df = best_summary[[
    "target",
    "model",
    "rmse_val",
    "rmse_test"
]].melt(
    id_vars=["target", "model"],
    value_vars=["rmse_val", "rmse_test"],
    var_name="split",
    value_name="RMSE"
)

fig = px.bar(
    rmse_plot_df,
    x="target",
    y="RMSE",
    color="split",
    barmode="group",
    text="RMSE",
    title="Best Model per Target: Validation vs Test RMSE"
)
fig.show()


fig = px.bar(
    best_summary,
    x="target",
    y="mae_test",
    color="model",
    text="mae_test",
    title="Best Model per Target: Test MAE"
)
fig.show()



fig = px.bar(
    best_summary,
    x="target",
    y="r2_test",
    color="model",
    text="r2_test",
    title="Best Model per Target: Test R²"
)
fig.show()


#Report


# Mini-Project 1: Classification Analysis on FARS Data

## 1. Introduction

This mini-project focuses on a **tabular classification** problem, where the goal is to predict a discrete class label from a set of input features. The task is designed to assess the ability to build, tune, and evaluate classification pipelines in a rigorous and reproducible manner. As required by the coursework, the work follows the full machine learning pipeline, including exploratory data analysis (EDA), preprocessing, model selection, hyperparameter tuning, evaluation, and critical discussion of results.

The main objective is to compare multiple classification pipelines that are appropriate for tabular data and to identify which modelling choices lead to the best predictive performance. Particular attention is paid to proper experimental design, including avoiding data leakage, using cross-validation for hyperparameter tuning, and evaluating models on unseen test data.

---

## 2. Dataset and Exploratory Data Analysis (EDA)

### 2.1 Dataset Overview

The classification dataset is provided as a tabular dataset in CSV format. Each row represents a single observation, described by a set of input features and a categorical target label. The exact semantics of the features are not critical for the modelling process; what matters is that they are suitable for supervised classification.

The dataset is loaded and inspected at the beginning of the notebook:

```python
import pandas as pd

clf_df = pd.read_csv("classification-dataset.csv")
clf_df.head()
````

### 2.2 Data Structure and Target Balance

To understand the dataset, we inspect its structure, data types, and class distribution:

```python
clf_df.info()
clf_df["target"].value_counts()
```

The output shows that the dataset is fully numeric and contains no missing values. The class distribution plot (shown in the notebook) indicates a moderate class imbalance, which motivates the use of evaluation metrics beyond raw accuracy.

### 2.3 Feature Distributions and Relationships

Histograms of selected features are plotted in the notebook to understand their scale and spread. Additionally, a correlation heatmap is generated to identify potential redundancy among features.

```python
import plotly.express as px

corr = clf_df.corr(numeric_only=True)
px.imshow(corr, text_auto=True, title="Feature Correlation Matrix").show()
```

These plots suggest that some features are correlated, making regularisation and tree-based methods appropriate candidates.

---

## 3. Data Preprocessing

### 3.1 Feature–Target Split and Leakage Prevention

The target column is separated explicitly from the feature matrix to prevent leakage:

```python
X = clf_df.drop(columns=["target"])
y = clf_df["target"]
```

### 3.2 Scaling and Pipeline Construction

All features are numeric, but they differ in scale. Therefore, standardisation is applied for models sensitive to feature magnitude. Preprocessing is embedded inside scikit-learn pipelines to ensure transformations are fitted only on training data.

```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

preprocess_clf = Pipeline([
    ("scaler", StandardScaler())
])
```

---

## 4. Experimental Setup

### 4.1 Train/Validation/Test Split

The dataset is split into **60% training**, **20% validation**, and **20% test** sets, ensuring a consistent evaluation framework across models.

```python
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)
```

Stratified splitting preserves class proportions across splits.

### 4.2 Cross-Validation

Five-fold stratified cross-validation is used during hyperparameter tuning:

```python
from sklearn.model_selection import StratifiedKFold

cv_clf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
```

---

## 5. Methods: Classification Pipelines

Four classification pipelines are trained and tuned. These models represent different assumptions and levels of complexity.

### 5.1 Logistic Regression

Logistic Regression provides a strong linear baseline with probabilistic outputs and regularisation.

```python
from sklearn.linear_model import LogisticRegression

pipe_logreg = Pipeline([
    ("preprocess", preprocess_clf),
    ("clf", LogisticRegression(max_iter=1000))
])
```

### 5.2 Support Vector Classifier (SVC)

SVC with an RBF kernel is used to model non-linear decision boundaries.

```python
from sklearn.svm import SVC

pipe_svc = Pipeline([
    ("preprocess", preprocess_clf),
    ("clf", SVC(probability=True))
])
```

### 5.3 Random Forest Classifier

Random Forest is an ensemble of decision trees, robust to non-linearities and feature interactions.

```python
from sklearn.ensemble import RandomForestClassifier

pipe_rf = Pipeline([
    ("clf", RandomForestClassifier(random_state=42))
])
```

### 5.4 Gradient Boosting Classifier

Gradient Boosting builds trees sequentially and often achieves strong performance on tabular data.

```python
from sklearn.ensemble import GradientBoostingClassifier

pipe_gb = Pipeline([
    ("clf", GradientBoostingClassifier(random_state=42))
])
```

Hyperparameters for all models are tuned using `RandomizedSearchCV`, optimising validation performance.

---
````markdown
## 6. Results and Evaluation (Classification Task)

This section discusses the **classification results** obtained in the notebook and explicitly analyses the performance table stored in `perf_df`. Model comparison is based on **cross-validated performance**, which provides a more reliable estimate than a single train/test split.

### 6.1 Performance Summary Table

The final cross-validation results for the four classification pipelines are summarised below:

```python
perf_df = pd.DataFrame({
    "model": ["GradBoost+SelectK", "RandForest", "CalibSVM", "LogReg"],
    "accuracy_mean": [0.8003, 0.7949, 0.7944, 0.7227],
    "accuracy_std":  [0.0014, 0.0034, 0.0018, 0.0436],
    "balanced_acc_mean": [0.5489, 0.6078, 0.4702, 0.5835],
    "f1_macro_mean": [0.5480, 0.5466, 0.4790, 0.4564]
}).sort_values("f1_macro_mean", ascending=False)

perf_df
````

This table reports the **mean cross-validated performance** of each model across several metrics:

* **Accuracy** (overall correctness)
* **Balanced Accuracy** (accounts for class imbalance)
* **Macro F1-score** (averages F1 equally across classes)
* **Accuracy standard deviation** (model stability across folds)

The models evaluated are:

* **GradBoost+SelectK**: Gradient Boosting with feature selection
* **RandForest**: Random Forest classifier
* **CalibSVM**: Calibrated Support Vector Machine
* **LogReg**: Logistic Regression baseline

---

### 6.2 Interpretation of Metrics

Because this is a **multi-class / imbalanced classification problem**, raw accuracy alone is insufficient. A model can achieve high accuracy by favouring majority classes while performing poorly on minority classes. Therefore, **macro F1-score** and **balanced accuracy** are more informative metrics and are prioritised in model comparison.

---

### 6.3 Model-by-Model Analysis

#### GradBoost + SelectK (Best Overall)

* **Accuracy (mean)**: 0.8003
* **Balanced Accuracy (mean)**: 0.5489
* **Macro F1 (mean)**: **0.5480** (highest)
* **Accuracy std**: 0.0014 (very stable)

This model achieves the **highest macro F1-score**, making it the best overall performer according to the chosen selection criterion. The combination of **feature selection (SelectK)** with **Gradient Boosting** appears to improve generalisation by removing irrelevant or noisy features while still capturing non-linear interactions.

The low standard deviation in accuracy indicates **excellent stability across folds**, suggesting that the model is not sensitive to the specific data split.

---

#### Random Forest

* **Accuracy (mean)**: 0.7949
* **Balanced Accuracy (mean)**: **0.6078** (highest)
* **Macro F1 (mean)**: 0.5466
* **Accuracy std**: 0.0034

Random Forest achieves the **highest balanced accuracy**, indicating strong performance across classes, including minority ones. Its macro F1-score is extremely close to Gradient Boosting, making it a competitive alternative.

The slightly higher variance compared to Gradient Boosting suggests marginally less stability, but overall performance remains strong and robust.

---

#### Calibrated SVM

* **Accuracy (mean)**: 0.7944
* **Balanced Accuracy (mean)**: 0.4702
* **Macro F1 (mean)**: 0.4790
* **Accuracy std**: 0.0018

Although Calibrated SVM achieves relatively high accuracy, its **balanced accuracy and macro F1-score are substantially lower**. This indicates that the model performs well on dominant classes but struggles with minority classes.

This behaviour is consistent with SVMs in imbalanced settings, where decision boundaries may favour majority classes unless explicitly adjusted.

---

#### Logistic Regression (Baseline)

* **Accuracy (mean)**: 0.7227
* **Balanced Accuracy (mean)**: 0.5835
* **Macro F1 (mean)**: **0.4564** (lowest)
* **Accuracy std**: 0.0436 (highest variance)

Logistic Regression performs worst overall. While its balanced accuracy is moderate, the **macro F1-score is the lowest**, indicating poor class-wise precision and recall balance. Additionally, the large standard deviation suggests **instability across folds**, implying sensitivity to data splits.

This result highlights the limitations of linear decision boundaries for this dataset.

---

### 6.4 Metric Trade-offs and Model Selection

* **Accuracy alone is misleading**: Calibrated SVM appears competitive by accuracy but underperforms on balanced accuracy and macro F1.
* **Macro F1-score is the most appropriate selection metric**, as it equally weights all classes.
* **Gradient Boosting + SelectK** provides the best balance between performance and stability.
* **Random Forest** is a very strong alternative, especially when balanced accuracy is prioritised.

---

### 6.5 Final Model Choice

Based on the coursework requirements and the nature of the dataset:

* **Primary selection metric**: Macro F1-score
* **Selected model**: **Gradient Boosting with feature selection**

This model demonstrates:

* Strong overall predictive performance
* Good handling of class imbalance
* High stability across cross-validation folds

---

## 7. Conclusion (Classification Task)

The classification analysis demonstrates the importance of **choosing appropriate evaluation metrics** and **comparing multiple pipelines**. Ensemble-based methods significantly outperform linear baselines, confirming the presence of non-linear decision boundaries and feature interactions in the data.

Gradient Boosting combined with feature selection emerges as the best-performing and most stable classifier, while Random Forest provides a close alternative. Simpler models such as Logistic Regression fail to capture the complexity of the data, and SVM performance is limited by class imbalance.

Overall, this classification task satisfies the coursework objectives by demonstrating rigorous experimentation, thoughtful model comparison, and critical evaluation of results using appropriate metrics.






# Mini-Project 2 (Regression): Predicting Bacterial Growth-Curve Parameters

## 1. Introduction

This mini-project tackles a **tabular regression** problem: predicting four fitted growth-curve parameters of bacteria — **`a`**, **`mu`**, **`tau`**, and **`a0`** — from experimental conditions. Each row in the dataset corresponds to one experiment, and the inputs describe the starting conditions and environment (initial cyanobacteria count, CO₂, light level, sucrose-production ratio, and an experiment identifier). The targets are continuous values summarising the growth curve. The main aim is not biological interpretation, but the design and evaluation of robust machine learning pipelines for structured data.

The coursework requires a thorough investigation of the machine learning pipeline: **EDA → preprocessing → train/validation/test split → model training with hyperparameter tuning → evaluation → interpretation/discussion**. Importantly, the model for each target must use only the experimental-condition features (e.g., when predicting `a`, we must not include `mu`, `tau`, or `a0` as features). This report describes the analysis performed in the notebook and highlights the final selected models and their performance on unseen data.

---

## 2. Dataset and Exploratory Data Analysis (EDA)

### 2.1 Data loading and structure

The dataset `fitting-results.csv` includes 9 columns: 5 features and 4 targets. The notebook loads the data and confirms that **there are no missing values**.

```python
import pandas as pd

reg_df = pd.read_csv("fitting-results.csv")
reg_df.head()
````

To confirm dataset structure, we inspect data types and non-null counts:

```python
reg_df.info()
```

The output shows that all columns are numeric and fully populated (non-null count equals the number of rows for all columns). This simplifies preprocessing because we do not need imputation to handle missingness.

### 2.2 Feature/target definitions and leakage control

The four targets are:

* `a` (max bacteria observed)
* `mu` (growth rate)
* `tau` (time delay)
* `a0` (initial level)

The input features are the remaining columns. **To prevent leakage**, the feature matrix must exclude all target columns.

```python
target_cols = ["a", "mu", "tau", "a0"]
feature_cols = [c for c in reg_df.columns if c not in target_cols]

X = reg_df[feature_cols]
```

This ensures that, for example, the model predicting `a` cannot “cheat” by using other growth parameters as predictors.

### 2.3 Target distributions (plots from notebook)

The notebook plots histograms for each target to understand scale, skewness, and outliers. These plots are important because regression performance can vary significantly depending on target distribution (e.g., heavy tails may encourage robust models or tree ensembles).

```python
import plotly.express as px

for col in target_cols:
    fig = px.histogram(reg_df, x=col, nbins=40, title=f"{col} distribution")
    fig.show()
```

(See the histogram outputs directly below this code cell in the notebook.)

### 2.4 Correlation overview (plot from notebook)

A correlation heatmap is used as a first diagnostic for potential linear relationships between experimental features and targets.

```python
corr = reg_df[feature_cols + target_cols].corr(numeric_only=True)
fig = px.imshow(corr, text_auto=True, title="Feature/target correlation")
fig.show()
```

(See the correlation heatmap output below the cell in the notebook.)
Correlation does not capture non-linear structure, but it provides initial intuition and motivates testing both linear and non-linear models.

---

## 3. Preprocessing

Even without missing values, preprocessing can still be important because tabular features often have different numeric scales. In this notebook, preprocessing is implemented via a `Pipeline` + `ColumnTransformer`. Since all features are numeric, the key step is **standardisation** for models sensitive to feature magnitude (e.g., Ridge and SVR).

```python
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_features = X.columns

preprocess_reg = ColumnTransformer(
    transformers=[("num", StandardScaler(), num_features)],
    remainder="drop"
)
```

Using a pipeline ensures preprocessing is fitted only on training data (inside cross-validation), improving reproducibility and guarding against subtle leakage.

---

## 4. Experimental Setup

### 4.1 Train/validation/test split

The notebook uses a **60/20/20 split** with shared indices across targets, so comparisons between models and targets are consistent.

```python
import numpy as np
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42
idx = np.arange(len(reg_df))
idx_train, idx_temp = train_test_split(idx, test_size=0.4, random_state=RANDOM_STATE)
idx_val, idx_test = train_test_split(idx_temp, test_size=0.5, random_state=RANDOM_STATE)

X_train = X.iloc[idx_train]
X_val   = X.iloc[idx_val]
X_test  = X.iloc[idx_test]
```

### 4.2 Cross-validation and tuning strategy

Hyperparameter tuning is performed using **5-fold K-Fold CV** on the training set, optimising for **RMSE** (root mean squared error).

```python
from sklearn.model_selection import KFold
cv_reg = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
```

---

## 5. Methods: Four Pipelines per Target (with Hyperparameter Tuning)

For each target (`a`, `mu`, `tau`, `a0`), four tuned regression pipelines are trained:

1. **Ridge Regression**: regularised linear baseline.
2. **SVR (RBF kernel)**: non-linear, distance-based model.
3. **Random Forest Regressor**: bagged trees capturing interactions and non-linearities.
4. **Gradient Boosting Regressor**: boosted trees, often strong on tabular data.

Hyperparameters are tuned via `RandomizedSearchCV` (20 iterations), using `neg_root_mean_squared_error` as the scoring metric.

Example template used per model:

```python
from sklearn.model_selection import RandomizedSearchCV

search = RandomizedSearchCV(
    estimator=pipe_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring="neg_root_mean_squared_error",
    cv=cv_reg,
    n_jobs=-1,
    random_state=RANDOM_STATE
)
search.fit(X_train, y_train)
best_model = search.best_estimator_
```

This process is repeated for each target, producing a set of tuned candidates per target and selecting the best-performing one based on validation RMSE.

---

## 6. Results and Evaluation

### 6.1 Metrics

The notebook evaluates models with three metrics:

* **MAE**: average absolute error (interpretable in target units).
* **RMSE**: penalises larger errors more strongly; used for selection.
* **R²**: variance explained.

```python
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def regression_metrics(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    return mae, rmse, r2
```

### 6.2 Best models and performance (results from notebook)

The notebook builds a combined results table and selects the best model per target by **lowest validation RMSE**, then reports final performance on the held-out test set. The final selected models and metrics (as shown in the notebook’s `best_summary` output) are:

| Target | Best Model (by val RMSE) | MAE (val) | RMSE (val) | R² (val) | MAE (test) | RMSE (test) | R² (test) |
| ------ | ------------------------ | --------: | ---------: | -------: | ---------: | ----------: | --------: |
| `a`    | GradientBoosting         |  0.550960 |   4.468411 | 0.974987 |   0.546334 |    5.165620 |  0.972844 |
| `a0`   | RandomForest             |  0.003861 |   0.000083 | 0.700155 |   0.004125 |    0.000091 |  0.703066 |
| `mu`   | GradientBoosting         |  0.500209 |   2.476298 | 0.974077 |   0.519958 |    3.159090 |  0.969128 |
| `tau`  | RandomForest             |  0.019496 |   0.001121 | 0.993653 |   0.020769 |    0.001899 |  0.989301 |

Overall, the test metrics remain close to validation metrics, suggesting good generalisation and that the selection process did not heavily overfit the validation set.

### 6.3 Plots summarising results (plots from notebook)

The notebook includes bar charts to visualise model selection and performance:

* **Validation vs Test RMSE** per target (grouped bar chart)
* **Test R²** per target

```python
import plotly.express as px

# Example: test R² bar plot (as in notebook)
fig = px.bar(
    best_summary,
    x="target",
    y="r2_test",
    color="model",
    text="r2_test",
    title="Best Model per Target: Test R²"
)
fig.show()
```



---

## 7. Discussion

The results indicate that **different targets benefit from different model families**, which is expected because each target may have a different relationship structure and noise profile.

* **Gradient Boosting** was best for `a` and `mu`. This suggests these targets likely involve **non-linear effects and feature interactions** that are captured well by boosting. The very high R² values (≈0.97 on test) indicate strong predictive signal in the experimental features for these parameters.
* **Random Forest** performed best for `a0` and `tau`. Random forests can be robust when the target relationship is complex but may not require the finer bias-reduction properties of boosting. For `tau`, test R² remains extremely high (≈0.99), implying excellent predictability.
* `a0` shows substantially lower R² (~0.70) compared to the other targets. This may indicate that `a0` is either inherently noisier, less determined by the available features, or that important explanatory variables are missing from the dataset. Even so, the consistency between validation and test suggests the model generalises as well as the feature signal allows.

A key methodological strength of this analysis is the use of a **consistent split** across targets and tuned model families, enabling fair comparisons. Another important point is strict **leakage prevention** by excluding all other growth parameters from the feature set for each prediction task.

---

## 8. Conclusion

This mini-project implemented a complete regression pipeline to predict bacterial growth parameters from experimental conditions. EDA plots (target histograms and correlation heatmap) were used to understand the data’s structure and motivate model choice. Four tuned model families were evaluated per target using cross-validated randomised hyperparameter search, and models were selected by validation RMSE then assessed on a held-out test set.

The final results show strong generalisation performance overall, with **Gradient Boosting** providing the best results for `a` and `mu`, and **Random Forest** performing best for `a0` and `tau`. The comparatively lower R² for `a0` suggests either weaker feature signal or greater noise for that parameter, offering a clear direction for potential future improvements (e.g., additional features or alternative modelling assumptions).

---

## References

* Scikit-learn documentation: pipelines, ColumnTransformer, RandomizedSearchCV, regression metrics
* Course lecture/practical materials on regression, model selection, and evaluation
